# Task 1: Three-layer feed-forward neural network

In [ ]:
import numpy as np
import pandas as pd
from random import randint
import matplotlib.pyplot as plt
from time import time 
import copy
import torch
import torch.nn as nn
import torch.nn.functional as F
from scipy.io import loadmat
import pdb
import torchvision
import torchvision.transforms as transforms
import os
import os.path
import sys
from PIL import Image
import sklearn
from sklearn.metrics import classification_report
from sklearn.utils import class_weight
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, TensorDataset



In [ ]:
device= torch.device("cuda")
#device= torch.device("cpu")
print(device)

cuda


In [ ]:
from google.colab import drive
drive.mount('/gdrive')

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


In [ ]:
# Hyper-parameters 
input_size = 2065
hidden_size = 500
mbatch_size = 64
TRAIN_SPLIT = 0.7

#epoch for now, set 2000/more for better answer
num_epochs = 3000
learning_rate = 0.001

histories={}
# DECLARATION OF CONSTANT VALUES

# for training of model
EPOCH_STARTER = 500 # print every 500 epoch for training of the models

# for plot size
WIDTH = 20
HEIGHT = 10

# optimizer
OPTIMIZERS = ['sgd']

# base path
COLAB_FILEPATH = '/gdrive/MyDrive/secondhalf_LAB/OQC.mat'

# sub-path
SUBPATH = 'saved_weights/'

In [ ]:
# scale data, delete if not necessary, try
def scale(X, X_min, X_max):
    return (X - X_min)/(X_max-X_min)

In [ ]:
#load matlab file 
oqc_dataset = loadmat(COLAB_FILEPATH)
oqc_dataset

{'__globals__': [],
 '__header__': b'MATLAB 5.0 MAT-file, Platform: PCWIN64, Created on: Sun Mar 29 14:29:37 2020',
 '__version__': '1.0',
 'data': array([[0.39052473, 0.14779758, 0.1563981 , ..., 0.07962529, 0.92246927,
         2.        ],
        [0.39052473, 0.14779758, 0.1563981 , ..., 0.05367681, 0.92148159,
         2.        ],
        [0.5084938 , 0.14779758, 0.1563981 , ..., 0.05611241, 0.9204939 ,
         2.        ],
        ...,
        [0.54001505, 0.00982888, 0.77251185, ..., 0.05611241, 0.93135813,
         0.        ],
        [0.60664394, 0.00982888, 0.77251185, ..., 0.05658079, 0.93086416,
         0.        ],
        [0.7189506 , 0.00982888, 0.77251185, ..., 0.05658079, 0.93086416,
         0.        ]])}

In [ ]:
data = oqc_dataset['data']
print(data.shape)

(2952, 49)


In [ ]:
# assign the corresponding part of the dataset to the predictor(X) and response(Y) variables 
X_raw, Y_raw = data[1:, :48].astype(np.float32), data[1:,-1].astype(np.long)
X = scale(X_raw, np.min(X_raw, axis=0), np.max(X_raw, axis=0))

Y = Y_raw.reshape(Y_raw.shape[0], 1)
#Y = Y.astype(np.long)

#perform train-test-split of the dataset
X_train, X_test, y_train, y_test = train_test_split(X, Y, train_size=TRAIN_SPLIT, shuffle=True, random_state=42)
X_train = torch.tensor(X_train)
X_test = torch.tensor(X_test)
y_train = torch.tensor(y_train)
y_test = torch.tensor(y_test)
n, c = X_train.shape


In [ ]:
class CustomTensorDataset(Dataset):
    """TensorDataset with support of transforms.
    """
    def __init__(self, tensors, transform=None):
        assert all(tensors[0].size(0) == tensor.size(0) for tensor in tensors)
        self.tensors = tensors
        self.transform = transform

    def __getitem__(self, index):
        x = self.tensors[0][index]

        if self.transform:
            x = self.transform(x)

        y = self.tensors[1][index]

        return x, y

    def __len__(self):
        return self.tensors[0].size(0)

In [ ]:
train_dataset_normal = CustomTensorDataset(tensors=(X_train, y_train), transform=None)
train_loader = torch.utils.data.DataLoader(train_dataset_normal, batch_size=mbatch_size)

test_dataset_normal = CustomTensorDataset(tensors=(X_test, y_test), transform=None)
test_loader = torch.utils.data.DataLoader(test_dataset_normal, batch_size=mbatch_size)

In [ ]:
# Fully 3-layer connected neural network with one hidden layer. Output includes as 1 layer
# Edit here to change 1. Hidden layer by adding nn.linear, 2. Change 512 (hidden nodes)
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(48, 8),
            nn.ReLU(),
            nn.Linear(8, 8),
            nn.ReLU(),
            nn.Linear(8, 3),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=48, out_features=8, bias=True)
    (1): ReLU()
    (2): Linear(in_features=8, out_features=8, bias=True)
    (3): ReLU()
    (4): Linear(in_features=8, out_features=3, bias=True)
  )
)


In [ ]:
# Loss and optimizer

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)  

In [ ]:
test_loss_plot = list()
correct=0
total=0

In [ ]:
# Train the model
#time0 = time.time()
def trainmodel(num_epochs):
  correct_count, all_count = 0, 0
  print(num_epochs)


  for epoch in range(num_epochs): 
    
      for i, (inputs, targets) in enumerate(train_loader):
        inputs = inputs.cuda()
        targets = targets.cuda()
        
        optimizer.zero_grad()
        # Forward pass
        outputs = model(inputs)
        #loss = criterion(outputs, torch.max(targets, 1)[1])
        loss = criterion(outputs, targets.squeeze(dim=1))
        
        # Backward and optimize
        loss.backward()
        optimizer.step()
        
        #_, predicted = outputs.max(1)
        #total += targets.size(0)
        #correct += predicted.eq(targets).sum().item()
        #rawPredicted, predicted = torch.max(F.softmax(inputs.data,dim=1), 1)
        #correct       = (predicted == targets).sum().item()
        #accuracy      = 100*correct/(predicted == targets).shape[0] 
        # ps = torch.exp(logps).cpu()
        # probab = list(ps.detach().numpy()[0])
        # pred_target = probab.index(max(probab))
        # true_target = targets.cpu().numpy()[i]
        # if(true_target == pred_target):
        #   correct_count += 1
        # all_count += 1
      
      # _, predicted = outputs.max(1)
        #total += targets.size(0)
        #correct += predicted.eq(targets).sum().item()
        #ps = torch.exp(logps).cpu()
        #probab = list(ps.detach().numpy()[0])
        #pred_target = probab.index(max(probab))
        #true_target = targets.cpu().numpy()[i]
        #if(true_target == pred_target):
        #  correct_count += 1
        #all_count += 1
      
        
      if (epoch+1) % 50 == 0:
        print ('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))

        
      
      test_loss_plot.append(loss.item())
      epochs_count = [i for i in range(num_epochs)]

  correct_count, all_count = 0, 0
  running_error = 0
  correct=0
  total=0

  with torch.no_grad():
    for i, (inputs, targets) in enumerate(test_loader):
      for i in range(len(targets)):
        
        inputs = inputs.cuda()
        targets = targets.cuda()

        logps = model(inputs)

        ps = torch.exp(logps).cpu()
        probab = list(ps.detach().numpy()[0])
        pred_target = probab.index(max(probab))
        true_target = targets.cpu().numpy()[i]
        if(true_target == pred_target):
          correct_count += 1
        all_count += 1
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()
        rawPredicted, predicted = torch.max(F.softmax(inputs.data,dim=1), 1)
        correct       = (predicted == targets).sum().item()
        accuracy      = 100*correct/(predicted == targets).shape[0] 


  print("Number Of Images Tested =", all_count)
  print("\nModel Accuracy =", (correct_count/all_count) * 100)
  print("Accuracy =", accuracy )
  #accu=100.*correct/total
  #print('Accuracy: %.3f'%(accu))

  #print("Number Of Images Tested =", all_count)
  #print("\nModel Accuracy =", (correct_count/all_count) * 100)
  #print("\nTraining Time (in minutes) =",(time.time()-time0)/60)


In [ ]:
epochlist = [1, 5, 10, 20, 30, 50, 100, 150, 200, 500, 1000, 1500, 2000]

for i in epochlist:
  trainmodel(i)

1
Number Of Images Tested = 886

Model Accuracy = 29.45823927765237
Accuracy = 1401.851851851852
5
Number Of Images Tested = 886

Model Accuracy = 29.45823927765237
Accuracy = 1401.851851851852
10
Number Of Images Tested = 886

Model Accuracy = 29.45823927765237
Accuracy = 1401.851851851852
20
Number Of Images Tested = 886

Model Accuracy = 29.45823927765237
Accuracy = 1401.851851851852
30
Number Of Images Tested = 886

Model Accuracy = 33.29571106094808
Accuracy = 1401.851851851852
50
Epoch [50/50], Loss: 1.1122
Number Of Images Tested = 886

Model Accuracy = 35.55304740406321
Accuracy = 1401.851851851852
100
Epoch [50/100], Loss: 1.1112
Epoch [100/100], Loss: 1.1003
Number Of Images Tested = 886

Model Accuracy = 37.24604966139955
Accuracy = 1401.851851851852
150
Epoch [50/150], Loss: 1.0918
Epoch [100/150], Loss: 1.0769
Epoch [150/150], Loss: 1.0468
Number Of Images Tested = 886

Model Accuracy = 37.24604966139955
Accuracy = 1401.851851851852
200
Epoch [50/200], Loss: 0.9867
Epoch [

KeyboardInterrupt: ignored

In [ ]:
# Plot the graph, find a better graph, optimally the loss aginst epoch graph, try to integrate
plt.figure(figsize=(WIDTH,HEIGHT))
#np.arrays(epoch_count)
#np.arrays(test_loss_plot)
plt.plot(epochs_count, test_loss_plot ,'green',label='test')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend(loc="best")
plt.show()

In [ ]:
# correct_count, all_count = 0, 0
# running_error = 0
# model.eval()

# correct=0
# total=0

# with torch.no_grad():
#   for i, (inputs, targets) in enumerate(train_loader):
#     for i in range(len(targets)):
      
#       inputs = inputs.cuda()
#       targets = targets.cuda()

#       logps = model(inputs)

#       ps = torch.exp(logps).cpu()
#       probab = list(ps.detach().numpy()[0])
#       pred_target = probab.index(max(probab))
#       true_target = targets.cpu().numpy()[i]
#       if(true_target == pred_target):
#         correct_count += 1
#       all_count += 1
#       _, predicted = outputs.max(1)
#       total += targets.size(0)
#       correct += predicted.eq(targets).sum().item()
#       rawPredicted, predicted = torch.max(F.softmax(inputs.data,dim=1), 1)
#       correct       = (predicted == targets).sum().item()
#       accuracy      = 100*correct/(predicted == targets).shape[0] 


# print("Number Of Images Tested =", all_count)
# print("\nModel Accuracy =", (correct_count/all_count) * 100)
# print("Accuracy =", accuracy )

In [ ]:
correct_count, all_count = 0, 0
running_error = 0
model.eval()

correct=0
total=0

with torch.no_grad():
  for i, (inputs, targets) in enumerate(test_loader):
    for i in range(len(targets)):
      
      inputs = inputs.cuda()
      targets = targets.cuda()

      logps = model(inputs)

      ps = torch.exp(logps).cpu()
      probab = list(ps.detach().numpy()[0])
      pred_target = probab.index(max(probab))
      true_target = targets.cpu().numpy()[i]
      if(true_target == pred_target):
        correct_count += 1
      all_count += 1
      _, predicted = outputs.max(1)
      total += targets.size(0)
      correct += predicted.eq(targets).sum().item()
      rawPredicted, predicted = torch.max(F.softmax(inputs.data,dim=1), 1)
      correct       = (predicted == targets).sum().item()
      accuracy      = 100*correct/(predicted == targets).shape[0] 


print("Number Of Images Tested =", all_count)
print("\nModel Accuracy =", (correct_count/all_count) * 100)
print("Accuracy =", accuracy )

In [ ]:
correct_count, all_count = 0, 0

for i, (inputs, targets) in enumerate(test_loader):
  for i in range(len(targets)):
      with torch.no_grad():
        # load data
        inputs   = inputs.to(device)
        targets  = targets.to(device)
        ones        = torch.tensor([1.0])
        targets  = targets.long()
    
        loss          = criterion(inputs,targets)
      rawPredicted, predicted = torch.max(F.softmax(inputs.data,dim=1), 1)
      correct       = (predicted == targets).sum().item()
      accuracy      = 100*correct/(predicted == targets).shape[0]  # 1: correct, 0: wrong

Accuracy = meanStd()
Accuracy.calcMeanStd(accuracy)
print("Number Of Images Tested =", all_count)
print('Accuracy: ',Accuracy.mean,'(+/-)',Accuracy.std)
  
  
  # net = net.eval()
  # with T.no_grad():
    # raw_out = net(unknown)    # a Tensor
  # pred_prob = raw_out.item()  # scalar, [0.0, 1.0]

In [ ]:
class meanStd(object):
    def __init__(self):
        self.mean     = 0.0
        self.mean_old = 0.0
        self.std      = 0.001
        self.count    = 0.0
        self.minMean  = 100.0
        self.minStd   = 100.0
        self.M_old    = 0.0
        self.M        = 0.0
        self.S        = 0.0
        self.S_old    = 0.0
        
    def calcMeanStd(self, data, cnt = 1):
        self.data     = data
        self.mean_old = copy.deepcopy(self.mean)
        self.M_old    = self.count*self.mean_old
        self.M        = self.M_old + data
        self.S_old    = copy.deepcopy(self.S)
        if self.count > 0:
            self.S    = self.S_old + ((self.count*data - self.M_old)**2)/(self.count*(self.count + cnt))
        
        self.count   += cnt
        self.mean     = self.mean_old + np.divide((data-self.mean_old),self.count)
        self.std      = np.sqrt(self.S/self.count)
        
        if (self.std != self.std).any():
            print('There is NaN in meanStd')
            pdb.set_trace()
    
    def resetMinMeanStd(self):
        self.minMean = copy.deepcopy(self.mean)
        self.minStd  = copy.deepcopy(self.std)
        
    def calcMeanStdMin(self):
        if self.mean < self.minMean:
            self.minMean = copy.deepcopy(self.mean)
        if self.std < self.minStd:
            self.minStd = copy.deepcopy(self.std)

# Task 2: Testing different network configuration

## 2.1 Test for variables in hidden nodes

In [ ]:
#Recall code for hidden node
hidden_node = 512
# Fully 3-layer connected neural network with one hidden layer. Output includes as 1 layer
# Edit here to change 1. Hidden layer by adding nn.linear, 2. Change 512 (hidden nodes)
#Reset list append
test_loss_plot = list()
list_test_loss_plot = list()

class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(48, hidden_node),
            nn.ReLU(),
            nn.Linear(hidden_node, hidden_node),
            nn.ReLU(),
            nn.Linear(hidden_node, 3),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)
print(model)


### Hidden nodes = 128

In [ ]:
#Test for different hidden nodes
# Loss and optimizer
train_dataset_normal = CustomTensorDataset(tensors=(X_train, y_train), transform=None)
train_loader = torch.utils.data.DataLoader(train_dataset_normal, batch_size=mbatch_size)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
# Train the model

num_epochs = 2000
total_step = len(X_train)
for epoch in range(num_epochs): 
  
    for i, (inputs, targets) in enumerate(train_loader):
      inputs = inputs.cuda()
      targets = targets.cuda()
      # Forward pass
      outputs = model(inputs)
      loss = criterion(outputs, targets.squeeze())
        
      # Backward and optimize
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()
                
    if (epoch+1) % 50 == 0:
      print ('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))
        
    test_loss_plot.append(loss.item())
    epochs_count = [i for i in range(num_epochs)]
list_test_loss_plot.append(test_loss_plot)

In [ ]:
# Plot the graph, find a better graph, optimally the loss aginst epoch graph, try to integrate
plt.figure(figsize=(WIDTH,HEIGHT))
plt.plot(epochs_count, list_test_loss_plot[0] ,'green',label='hn = 128')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend(loc="best")
plt.show()

### Hidden nodes = 256

In [ ]:
#Test for different hidden nodes
hidden_node = 256
train_dataset_normal = CustomTensorDataset(tensors=(X_train, y_train), transform=None)
train_loader = torch.utils.data.DataLoader(train_dataset_normal, batch_size=mbatch_size)
# Loss and optimizer
model = NeuralNetwork().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
# Train the model

test_loss_plot = list()
num_epochs = 2000
for epoch in range(num_epochs): 
  
    for i, (inputs, targets) in enumerate(train_loader):
      inputs = inputs.cuda()
      targets = targets.cuda()
      # Forward pass
      outputs = model(inputs)
      loss = criterion(outputs, targets.squeeze())
        
      # Backward and optimize
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()
                
    if (epoch+1) % 50 == 0:
      print ('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))
        
    test_loss_plot.append(loss.item())
    epochs_count = [i for i in range(num_epochs)]
list_test_loss_plot.append(test_loss_plot)



In [ ]:
# Plot the graph, find a better graph, optimally the loss aginst epoch graph, try to integrate
plt.figure(figsize=(WIDTH,HEIGHT))
plt.plot(epochs_count, list_test_loss_plot[1] ,'green',label='hn = 256')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend(loc="best")
plt.show()

### Hidden nodes = 512

In [ ]:
#Test for different hidden nodes
hidden_node = 512
train_dataset_normal = CustomTensorDataset(tensors=(X_train, y_train), transform=None)
train_loader = torch.utils.data.DataLoader(train_dataset_normal, batch_size=mbatch_size)
# Loss and optimizer
model = NeuralNetwork().to(device)
criterion = nn.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
# Train the model

num_epochs = 2000
test_loss_plot = list()
for epoch in range(num_epochs): 
  
    for i, (inputs, targets) in enumerate(train_loader):
      inputs = inputs.cuda()
      targets = targets.cuda()
      # Forward pass
      outputs = model(inputs)
      loss = criterion(outputs, targets.squeeze())
        
      # Backward and optimize
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()
                
    if (epoch+1) % 50 == 0:
      print ('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))
        
    test_loss_plot.append(loss.item())
    epochs_count = [i for i in range(num_epochs)]
list_test_loss_plot.append(test_loss_plot)

### Hidden nodes = 1024

In [ ]:
#Test for different hidden nodes
hidden_node = 1024
train_dataset_normal = CustomTensorDataset(tensors=(X_train, y_train), transform=None)
train_loader = torch.utils.data.DataLoader(train_dataset_normal, batch_size=mbatch_size)
# Loss and optimizer
model = NeuralNetwork().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
# Train the model

num_epochs = 2000
test_loss_plot = list()
for epoch in range(num_epochs): 
  
    for i, (inputs, targets) in enumerate(train_loader):
      inputs = inputs.cuda()
      targets = targets.cuda()
      # Forward pass
      outputs = model(inputs)
      loss = criterion(outputs, targets.squeeze())
        
      # Backward and optimize
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()
                
    if (epoch+1) % 50 == 0:
      print ('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))
        
    test_loss_plot.append(loss.item())
    epochs_count = [i for i in range(num_epochs)]
list_test_loss_plot.append(test_loss_plot)

In [ ]:
# Plot the graph, find a better graph, optimally the loss aginst epoch graph, try to integrate
plt.figure(figsize=(WIDTH,HEIGHT))
plt.plot(epochs_count, list_test_loss_plot[3] ,'green',label='hn = 1024')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend(loc="best")
plt.show()

### Hidden nodes = 2048

In [ ]:
 #Test for different hidden nodes
hidden_node = 2048
train_dataset_normal = CustomTensorDataset(tensors=(X_train, y_train), transform=None)
train_loader = torch.utils.data.DataLoader(train_dataset_normal, batch_size=mbatch_size)
# Loss and optimizer
model = NeuralNetwork().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
# Train the model

num_epochs = 2000
test_loss_plot = list()
for epoch in range(num_epochs): 
  
    for i, (inputs, targets) in enumerate(train_loader):
      inputs = inputs.cuda()
      targets = targets.cuda()
      # Forward pass
      outputs = model(inputs)
      loss = criterion(outputs, targets.squeeze())
        
      # Backward and optimize
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()
                
    if (epoch+1) % 50 == 0:
      print ('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))
        
    test_loss_plot.append(loss.item())
    epochs_count = [i for i in range(num_epochs)]
list_test_loss_plot.append(test_loss_plot)

In [ ]:
# Plot the graph, find a better graph, optimally the loss aginst epoch graph, try to integrate
plt.figure(figsize=(WIDTH,HEIGHT))
plt.plot(epochs_count, list_test_loss_plot[4] ,'green',label='hn = 2048')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend(loc="best")
plt.show()

### Comparing all different hidden nodes against test loss

In [ ]:
# cross validation accuracy of different hidden neuron against epochs
plt.figure(figsize=(WIDTH,HEIGHT))
plt.title("Loss vs Epochs")
plt.plot(epochs_count,list_test_loss_plot[0],'yellow',label='hidden nodes = 128')
plt.plot(epochs_count,list_test_loss_plot[1],'orange',label='hidden nodes = 256')
plt.plot(epochs_count,list_test_loss_plot[2],'green',label='hidden nodes = 512')
plt.plot(epochs_count,list_test_loss_plot[3],'blue',label='hidden nodes = 1024')
plt.plot(epochs_count,list_test_loss_plot[4],'red',label='hidden nodes = 2048')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend(loc="best")

## 2.2 Test for variables in hidden layers

### Hidden layers = 0

In [ ]:
#Recall code for hidden node

# Fully 3-layer connected neural network with one hidden layer. Output includes as 1 layer
# Edit here to change 1. Hidden layer by adding nn.linear, 2. Change 512 (hidden nodes)
hidden_node = 512
#Reset list append
test_loss_plot = list()
list_test_loss_plot = list()

class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(48, hidden_node),
            nn.ReLU(),
            nn.Linear(hidden_node, 3),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)
print(model)



In [ ]:
#For test, put to 2000 later
num_epochs = 2000
train_dataset_normal = CustomTensorDataset(tensors=(X_train, y_train), transform=None)
train_loader = torch.utils.data.DataLoader(train_dataset_normal, batch_size=mbatch_size)
# Loss and optimizer
model = NeuralNetwork().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)  
#Reset test_loss_plot
test_loss_plot = list()

# Train the model
for epoch in range(num_epochs): 
  
    for i, (inputs, targets) in enumerate(train_loader):
      inputs = inputs.cuda()
      targets = targets.cuda()
      # Forward pass
      outputs = model(inputs)
      loss = criterion(outputs, targets.squeeze())
        
      # Backward and optimize
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()
                
    if (epoch+1) % 50 == 0:
      print ('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))
        
    test_loss_plot.append(loss.item())
    epochs_count = [i for i in range(num_epochs)]
list_test_loss_plot.append(test_loss_plot)

In [ ]:
# Plot the graph, find a better graph, optimally the loss aginst epoch graph, try to integrate
plt.figure(figsize=(WIDTH,HEIGHT))
plt.plot(epochs_count, list_test_loss_plot[0] ,'green',label='hd = 0')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend(loc="best")
plt.show()

### Hidden layers = 1

In [ ]:
#Recall code for hidden node

# Fully 3-layer connected neural network with one hidden layer. Output includes as 1 layer
# Edit here to change 1. Hidden layer by adding nn.linear, 2. Change 512 (hidden nodes)
hidden_node = 512

class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(48, hidden_node),
            nn.ReLU(),
            nn.Linear(hidden_node, hidden_node),
            nn.ReLU(),
            nn.Linear(hidden_node, 3),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)
print(model)


In [ ]:
#For test, put to 2000 later
num_epochs = 2000
train_dataset_normal = CustomTensorDataset(tensors=(X_train, y_train), transform=None)
train_loader = torch.utils.data.DataLoader(train_dataset_normal, batch_size=mbatch_size)
# Loss and optimizer
model = NeuralNetwork().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)  
#Reset test_loss_plot
test_loss_plot = list()

# Train the model
for epoch in range(num_epochs): 
  
    for i, (inputs, targets) in enumerate(train_loader):
      inputs = inputs.cuda()
      targets = targets.cuda()
      # Forward pass
      outputs = model(inputs)
      loss = criterion(outputs, targets.squeeze())
        
      # Backward and optimize
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()
                
    if (epoch+1) % 50 == 0:
      print ('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))
        
    test_loss_plot.append(loss.item())
    epochs_count = [i for i in range(num_epochs)]

In [ ]:
# Plot the graph, find a better graph, optimally the loss aginst epoch graph, try to integrate
plt.figure(figsize=(WIDTH,HEIGHT))
plt.plot(epochs_count, list_test_loss_plot[1] ,'green',label='hd = 1')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend(loc="best")
plt.show()

### Hidden layers = 2

In [ ]:
#Recall code for hidden node

# Fully 3-layer connected neural network with one hidden layer. Output includes as 1 layer
# Edit here to change 1. Hidden layer by adding nn.linear, 2. Change 512 (hidden nodes)
hidden_node = 512

class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(48, hidden_node),
            nn.ReLU(),
            nn.Linear(hidden_node, hidden_node),
            nn.ReLU(),
            nn.Linear(hidden_node, hidden_node),
            nn.ReLU(),
            nn.Linear(hidden_node, 3),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)
print(model)

 

In [ ]:
#For test, put to 2000 later
num_epochs = 2000
train_dataset_normal = CustomTensorDataset(tensors=(X_train, y_train), transform=None)
train_loader = torch.utils.data.DataLoader(train_dataset_normal, batch_size=mbatch_size)
# Loss and optimizer
model = NeuralNetwork().to(device)
criterion = nnn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate) 
#Reset test_loss_plot
test_loss_plot = list()

# Train the model
for epoch in range(num_epochs): 
  
    for i, (inputs, targets) in enumerate(train_loader):
      inputs = inputs.cuda()
      targets = targets.cuda()
      # Forward pass
      loss = criterion(outputs, targets.squeeze())
        
      # Backward and optimize
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()
                
    if (epoch+1) % 50 == 0:
      print ('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))
        
    test_loss_plot.append(loss.item())
    epochs_count = [i for i in range(num_epochs)]
list_test_loss_plot.append(test_loss_plot)

In [ ]:
# Plot the graph, find a better graph, optimally the loss aginst epoch graph, try to integrate
plt.figure(figsize=(WIDTH,HEIGHT))
plt.plot(epochs_count, list_test_loss_plot[2] ,'green',label='hd = 2')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend(loc="best")
plt.show()

### Hidden layers = 3

In [ ]:
#Recall code for hidden node

# Fully 3-layer connected neural network with one hidden layer. Output includes as 1 layer
# Edit here to change 1. Hidden layer by adding nn.linear, 2. Change 512 (hidden nodes)
hidden_node = 512

class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(48, hidden_node),
            nn.ReLU(),
            nn.Linear(hidden_node, hidden_node),
            nn.ReLU(),
            nn.Linear(hidden_node, hidden_node),
            nn.ReLU(),
            nn.Linear(hidden_node, hidden_node),
            nn.ReLU(),
            nn.Linear(hidden_node, 3),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)
print(model)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)  

In [ ]:
#For test, put to 2000 later
num_epochs = 2000
train_dataset_normal = CustomTensorDataset(tensors=(X_train, y_train), transform=None)
train_loader = torch.utils.data.DataLoader(train_dataset_normal, batch_size=mbatch_size)
# Loss and optimizer
model = NeuralNetwork().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate) 
#Reset test_loss_plot
test_loss_plot = list()

# Train the model
for epoch in range(num_epochs): 
  
    for i, (inputs, targets) in enumerate(train_loader):
      inputs = inputs.cuda()
      targets = targets.cuda()
      # Forward pass
      outputs = model(inputs)
      #loss = criterion(outputs, torch.max(targets, 1)[1])
      loss = criterion(outputs, targets.squeeze())
        
      # Backward and optimize
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()
                
    if (epoch+1) % 50 == 0:
      print ('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))
        
    test_loss_plot.append(loss.item())
    epochs_count = [i for i in range(num_epochs)]
list_test_loss_plot.append(test_loss_plot)


In [ ]:
# Plot the graph, find a better graph, optimally the loss aginst epoch graph, try to integrate
plt.figure(figsize=(WIDTH,HEIGHT))
plt.plot(epochs_count, list_test_loss_plot[3] ,'green',label='hd = 3')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend(loc="best")
plt.show()

### Hidden layers = 4

In [ ]:
#Recall code for hidden node

# Fully 3-layer connected neural network with one hidden layer. Output includes as 1 layer
# Edit here to change 1. Hidden layer by adding nn.linear, 2. Change 512 (hidden nodes)
hidden_node = 512

class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(48, hidden_node),
            nn.ReLU(),
            nn.Linear(hidden_node, hidden_node),
            nn.ReLU(),
            nn.Linear(hidden_node, hidden_node),
            nn.ReLU(),
            nn.Linear(hidden_node, hidden_node),
            nn.ReLU(),
            nn.Linear(hidden_node, hidden_node),
            nn.ReLU(),
            nn.Linear(hidden_node, 3),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)
print(model)



In [ ]:
#For test, put to 2000 later
num_epochs = 2000
train_dataset_normal = CustomTensorDataset(tensors=(X_train, y_train), transform=None)
train_loader = torch.utils.data.DataLoader(train_dataset_normal, batch_size=mbatch_size)
# Loss and optimizer
model = NeuralNetwork().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)  
#Reset test_loss_plot
test_loss_plot = list()

# Train the model
num_epochs = 2000
for epoch in range(num_epochs): 
  
    for i, (inputs, targets) in enumerate(train_loader):
      inputs = inputs.cuda()
      targets = targets.cuda()
      # Forward pass
      outputs = model(inputs)
      loss = criterion(outputs, targets.squeeze())
        
      # Backward and optimize
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()
                
    if (epoch+1) % 50 == 0:
      print ('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))
        
    test_loss_plot.append(loss.item())
    epochs_count = [i for i in range(num_epochs)]
list_test_loss_plot.append(test_loss_plot)

In [ ]:
# Plot the graph, find a better graph, optimally the loss aginst epoch graph, try to integrate
plt.figure(figsize=(WIDTH,HEIGHT))
plt.plot(epochs_count, list_test_loss_plot[4] ,'green',label='hd = 4')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend(loc="best")
plt.show()

### Comparing all different hidden layers against test loss

In [ ]:
# cross validation accuracy of different hidden neuron against epochs
plt.figure(figsize=(WIDTH,HEIGHT))
plt.title("Loss vs Epochs")
plt.plot(epochs_count,list_test_loss_plot[0],'yellow',label='hidden layers = 0')
plt.plot(epochs_count,list_test_loss_plot[1],'orange',label='hidden layers = 1')
plt.plot(epochs_count,list_test_loss_plot[2],'green',label='hidden layers = 2')
plt.plot(epochs_count,list_test_loss_plot[3],'blue',label='hidden layers = 3')
plt.plot(epochs_count,list_test_loss_plot[4],'red',label='hidden layers = 4')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend(loc="best")

# Task 3: Testing different learning rates and setting adaptive learning rate to different epochs

## 3.1 Experiment with different learning rates

### Learning rate = 0

In [ ]:
# Fully 3-layer connected neural network with one hidden layer. Output includes as 1 layer
# Edit here to change 1. Hidden layer by adding nn.linear, 2. Change 512 (hidden nodes)

#Reset list append
test_loss_plot = list()
list_test_loss_plot = list()

class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(48, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 3),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)
print(model)

In [ ]:
#Test for different hidden nodes
learning_rate = 0
train_dataset_normal = CustomTensorDataset(tensors=(X_train, y_train), transform=None)
train_loader = torch.utils.data.DataLoader(train_dataset_normal, batch_size=mbatch_size)
# Loss and optimizer
model = NeuralNetwork().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)  
#For test, put to 2000 later
num_epochs = 2000
#Reset test_loss_plot
test_loss_plot = list()

# Train the model
for epoch in range(num_epochs): 
  
    for i, (inputs, targets) in enumerate(train_loader):
      inputs = inputs.cuda()
      targets = targets.cuda()
      # Forward pass
      outputs = model(inputs)
      #loss = criterion(outputs, torch.max(targets, 1)[1])
      loss = criterion(outputs, targets.squeeze())
        
      # Backward and optimize
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()

    if (epoch+1) % 50 == 0:
      print ('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))
        
    test_loss_plot.append(loss.item())
    epochs_count = [i for i in range(num_epochs)]

list_test_loss_plot.append(test_loss_plot)

In [ ]:
# Plot the graph, find a better graph, optimally the loss aginst epoch graph, try to integrate
plt.figure(figsize=(WIDTH,HEIGHT))
plt.plot(epochs_count, list_test_loss_plot[0] ,'green',label='lr = 0')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend(loc="best")
plt.show()

### Learning rate = 10**(-1)

In [ ]:
#Test for different hidden nodes
learning_rate = 10**(-1)
train_dataset_normal = CustomTensorDataset(tensors=(X_train, y_train), transform=None)
train_loader = torch.utils.data.DataLoader(train_dataset_normal, batch_size=mbatch_size)
# Loss and optimizer
model = NeuralNetwork().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)  
#For test, put to 2000 later
num_epochs = 2000
#Reset test_loss_plot
test_loss_plot = list()

# Train the model
for epoch in range(num_epochs): 
  
    for i, (inputs, targets) in enumerate(train_loader):
      inputs = inputs.cuda()
      targets = targets.cuda()
      # Forward pass
      outputs = model(inputs)
      #loss = criterion(outputs, torch.max(targets, 1)[1])
      loss = criterion(outputs, targets.squeeze())
        
      # Backward and optimize
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()
                
    if (epoch+1) % 50 == 0:
      print ('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))
        
    test_loss_plot.append(loss.item())
    epochs_count = [i for i in range(num_epochs)]
list_test_loss_plot.append(test_loss_plot)


In [ ]:
# Plot the graph, find a better graph, optimally the loss aginst epoch graph, try to integrate
plt.figure(figsize=(WIDTH,HEIGHT))
plt.plot(epochs_count, list_test_loss_plot[1] ,'green',label='lr = 10**(-1)')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend(loc="best")
plt.show()

### Learning rate = 10**(-2)

In [ ]:
#Test for different hidden nodes
learning_rate = 10**(-2)
train_dataset_normal = CustomTensorDataset(tensors=(X_train, y_train), transform=None)
train_loader = torch.utils.data.DataLoader(train_dataset_normal, batch_size=mbatch_size)
# Loss and optimizer
model = NeuralNetwork().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)  
#For test, put to 2000 later
num_epochs = 2000
#Reset test_loss_plot
test_loss_plot = list()

# Train the model
for epoch in range(num_epochs): 
  
    for i, (inputs, targets) in enumerate(train_loader):
      inputs = inputs.cuda()
      targets = targets.cuda()
      # Forward pass
      outputs = model(inputs)
      #loss = criterion(outputs, torch.max(targets, 1)[1])
      loss = criterion(outputs, targets.squeeze())
        
      # Backward and optimize
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()
                
    if (epoch+1) % 50 == 0:
      print ('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))
        
    test_loss_plot.append(loss.item())
    epochs_count = [i for i in range(num_epochs)]
list_test_loss_plot.append(test_loss_plot)

In [ ]:
# Plot the graph, find a better graph, optimally the loss aginst epoch graph, try to integrate
plt.figure(figsize=(WIDTH,HEIGHT))
plt.plot(epochs_count, list_test_loss_plot[2] ,'green',label='lr = 10**(-2)')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend(loc="best")
plt.show()

### Learning rate = 10**(-3)

In [ ]:
#Test for different hidden nodes
learning_rate = 10**(-3)
train_dataset_normal = CustomTensorDataset(tensors=(X_train, y_train), transform=None)
train_loader = torch.utils.data.DataLoader(train_dataset_normal, batch_size=mbatch_size)
# Loss and optimizer
model = NeuralNetwork().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)  
#For test, put to 2000 later
num_epochs = 2000
#Reset test_loss_plot
test_loss_plot = list()


# Train the model
for epoch in range(num_epochs): 
  
    for i, (inputs, targets) in enumerate(train_loader):
      inputs = inputs.cuda()
      targets = targets.cuda()
      # Forward pass
      outputs = model(inputs)
      #loss = criterion(outputs, torch.max(targets, 1)[1])
      loss = criterion(outputs, targets.squeeze())
        
      # Backward and optimize
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()
                
    if (epoch+1) % 50 == 0:
      print ('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))
        
    test_loss_plot.append(loss.item())
    epochs_count = [i for i in range(num_epochs)]
list_test_loss_plot.append(test_loss_plot)

In [ ]:
# Plot the graph, find a better graph, optimally the loss aginst epoch graph, try to integrate
plt.figure(figsize=(WIDTH,HEIGHT))
plt.plot(epochs_count, list_test_loss_plot[3] ,'green',label='lr = 10**(-3)')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend(loc="best")
plt.show()

### Learning rate = 10**(-4)

In [ ]:
#Test for different hidden nodes
learning_rate = 10**(-4)
train_dataset_normal = CustomTensorDataset(tensors=(X_train, y_train), transform=None)
train_loader = torch.utils.data.DataLoader(train_dataset_normal, batch_size=mbatch_size)
# Loss and optimizer
model = NeuralNetwork().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)  
#For test, put to 2000 later
num_epochs = 2000
#Reset test_loss_plot
test_loss_plot = list()


# Train the model
for epoch in range(num_epochs): 
  
    for i, (inputs, targets) in enumerate(train_loader):
      inputs = inputs.cuda()
      targets = targets.cuda()
      # Forward pass
      outputs = model(inputs)
      #loss = criterion(outputs, torch.max(targets, 1)[1])
      loss = criterion(outputs, targets.squeeze())
        
      # Backward and optimize
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()
                
    if (epoch+1) % 50 == 0:
      print ('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))
        
    test_loss_plot.append(loss.item())
    epochs_count = [i for i in range(num_epochs)]
list_test_loss_plot.append(test_loss_plot)

In [ ]:
# Plot the graph, find a better graph, optimally the loss aginst epoch graph, try to integrate
plt.figure(figsize=(WIDTH,HEIGHT))
plt.plot(epochs_count, list_test_loss_plot[4] ,'green',label='lr = 10**(-4)')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend(loc="best")
plt.show()

### Comparing all different learning rates against test loss

In [ ]:
# cross validation accuracy of different hidden neuron against epochs
plt.figure(figsize=(WIDTH,HEIGHT))
plt.title("Loss vs Epochs")
plt.plot(epochs_count,list_test_loss_plot[0],'yellow',label='lr = 0')
plt.plot(epochs_count,list_test_loss_plot[1],'orange',label='lr = 10**(-1)')
plt.plot(epochs_count,list_test_loss_plot[2],'green',label='lr = 10**(-2)')
plt.plot(epochs_count,list_test_loss_plot[3],'blue',label='lr = 10**(-3)')
plt.plot(epochs_count,list_test_loss_plot[4],'red',label='lr = 10**(-4)')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend(loc="best")

## 3.2 Experimenting with optimal learning rate but different epochs

### We can take learning rate = 10**(-1) and change the epochs to decide the adaptive learning rates to number of epochs

In [ ]:
#Reset list append
test_loss_plot = list()
list_test_loss_plot = list()

#Test for different hidden nodes
learning_rate = 10**(-1)
train_dataset_normal = CustomTensorDataset(tensors=(X_train, y_train), transform=None)
train_loader = torch.utils.data.DataLoader(train_dataset_normal, batch_size=mbatch_size)
# Loss and optimizer
model = NeuralNetwork().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)  
#For test, put to 3000 later
num_epochs = 3000
#Reset test_loss_plot
test_loss_plot = list()

# Train the model
for epoch in range(num_epochs): 
  
    for i, (inputs, targets) in enumerate(train_loader):
      inputs = inputs.cuda()
      targets = targets.cuda()
      # Forward pass
      outputs = model(inputs)
      #loss = criterion(outputs, torch.max(targets, 1)[1])
      loss = criterion(outputs, targets.squeeze())
        
      # Backward and optimize
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()
                
    if (epoch+1) % 50 == 0:
      print ('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))
        
    test_loss_plot.append(loss.item())
    epochs_count = [i for i in range(num_epochs)]
list_test_loss_plot.append(test_loss_plot)

In [ ]:
# Plot the graph, find a better graph, optimally the loss aginst epoch graph, try to integrate
plt.figure(figsize=(WIDTH,HEIGHT))
plt.plot(epochs_count, list_test_loss_plot[0] ,'green',label='Epoch = 2000')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend(loc="best")
plt.show()

In [ ]:
#Test for different hidden nodes
learning_rate = 10**(-1)
# Loss and optimizer
model = NeuralNetwork().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)  
#For test, put to 4000 later
num_epochs = 4000
#Reset test_loss_plot
test_loss_plot = list()

# Train the model
total_step = len(X_train)
for epoch in range(num_epochs): 
    for i in range((n - 1) // mbatch_size + 1):
      start_i = i * mbatch_size
      end_i = start_i + mbatch_size
      inputs = torch.from_numpy(X_train[start_i:end_i])
      targets = torch.from_numpy(y_train[start_i:end_i])

      # Forward pass
      outputs = model(inputs)
      loss = criterion(outputs, targets)
        
      # Backward and optimize
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()
                
    if (epoch+1) % 50 == 0:
      print ('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))
        
    test_loss_plot.append(loss.item())

    epochs_count = [i for i in range(num_epochs)]
    
list_test_loss_plot.append(test_loss_plot)

In [ ]:
# Plot the graph, find a better graph, optimally the loss aginst epoch graph, try to integrate
plt.figure(figsize=(WIDTH,HEIGHT))
plt.plot(epochs_count, list_test_loss_plot[1] ,'green',label='Epoch = 3000')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend(loc="best")
plt.show()

In [ ]:
#Test for different hidden nodes
learning_rate = 10**(-1)
# Loss and optimizer
model = NeuralNetwork().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)  
#For test, put to 5000 later
num_epochs = 5000
#Reset test_loss_plot
test_loss_plot = list()

# Train the model
total_step = len(X_train)
for epoch in range(num_epochs): 
    # Convert numpy arrays to torch tensors
    inputs = torch.from_numpy(X_train)
    targets = torch.from_numpy(y_train)
        
    # Forward pass
    outputs = model(inputs)
    loss = criterion(outputs, targets)
        
    # Backward and optimize
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
        
    if (epoch+1) % 50 == 0:
        print ('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))
    test_loss_plot.append(loss.item())

    epochs_count = [i for i in range(num_epochs)]

list_test_loss_plot.append(test_loss_plot)


In [ ]:
# Plot the graph, find a better graph, optimally the loss aginst epoch graph, try to integrate
plt.figure(figsize=(WIDTH,HEIGHT))
plt.plot(epochs_count, list_test_loss_plot[2] ,'green',label='Epoch = 4000')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend(loc="best")
plt.show()

In [ ]:
#Test for different hidden nodes
learning_rate = 10**(-1)
# Loss and optimizer
model = NeuralNetwork().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)  
#For test, put to 6000 later
num_epochs = 6000
#Reset test_loss_plot
test_loss_plot = list()

# Train the model
total_step = len(X_train)
for epoch in range(num_epochs): 
    # Convert numpy arrays to torch tensors
    inputs = torch.from_numpy(X_train)
    targets = torch.from_numpy(y_train)
        
    # Forward pass
    outputs = model(inputs)
    loss = criterion(outputs, targets)
        
    # Backward and optimize
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
        
    if (epoch+1) % 50 == 0:
        print ('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))
    test_loss_plot.append(loss.item())

    epochs_count = [i for i in range(num_epochs)]

list_test_loss_plot.append(test_loss_plot)


In [ ]:
# Plot the graph, find a better graph, optimally the loss aginst epoch graph, try to integrate
plt.figure(figsize=(WIDTH,HEIGHT))
plt.plot(epochs_count, list_test_loss_plot[3] ,'green',label='Epoch = 2000')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend(loc="best")
plt.show()

In [ ]:
# cross validation accuracy of different hidden neuron against epochs
plt.figure(figsize=(WIDTH,HEIGHT))
plt.title("Loss vs Epochs")
plt.plot(epochs_count,list_test_loss_plot[0],'yellow',label='Epoch = 2000')
plt.plot(epochs_count,list_test_loss_plot[1],'orange',label='Epoch = 3000')
plt.plot(epochs_count,list_test_loss_plot[2],'green',label='Epoch = 4000')
plt.plot(epochs_count,list_test_loss_plot[3],'blue',label='Epoch = 2000')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend(loc="best")

# Task 4: Testing different mini-batch size

### Mini-batch size = 1

In [ ]:
#Recall code for hidden node

# Fully 3-layer connected neural network with one hidden layer. Output includes as 1 layer
# Edit here to change 1. Hidden layer by adding nn.linear, 2. Change 512 (hidden nodes)
hidden_node = 512
#Reset list append
test_loss_plot = list()
list_test_loss_plot = list()

class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(48, hidden_node),
            nn.ReLU(),
            nn.Linear(hidden_node, hidden_node),
            nn.ReLU(),
            nn.Linear(hidden_node, 3),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)
print(model)


In [ ]:
 #Test for different batch size
mbatch_size = 1
train_dataset_normal = CustomTensorDataset(tensors=(X_train, y_train), transform=None)
train_loader = torch.utils.data.DataLoader(train_dataset_normal, batch_size=mbatch_size)
# Loss and optimizer
model = NeuralNetwork().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
#For test, put to 2000 later
num_epochs = 2000
#Reset test_loss_plot
test_loss_plot = list()
list_test_loss_plot = list()

# Train the model
for epoch in range(num_epochs): 
  
    for i, (inputs, targets) in enumerate(train_loader):
      inputs = inputs.cuda()
      targets = targets.cuda()
      # Forward pass
      outputs = model(inputs)
      #loss = criterion(outputs, torch.max(targets, 1)[1])
      loss = criterion(outputs, targets.squeeze())
        
      # Backward and optimize
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()
                
    if (epoch+1) % 50 == 0:
      print ('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))
        
    test_loss_plot.append(loss.item())
    epochs_count = [i for i in range(num_epochs)]
list_test_loss_plot.append(test_loss_plot)


In [ ]:
# Plot the graph, find a better graph, optimally the loss aginst epoch graph, try to integrate
plt.figure(figsize=(WIDTH,HEIGHT))
plt.plot(epochs_count, list_test_loss_plot[0] ,'green',label='mbs = 1')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend(loc="best")
plt.show()

### Mini-batch size = 8

In [ ]:
 #Test for different batch size
mbatch_size = 8
train_dataset_normal = CustomTensorDataset(tensors=(X_train, y_train), transform=None)
train_loader = torch.utils.data.DataLoader(train_dataset_normal, batch_size=mbatch_size)
# Loss and optimizer
model = NeuralNetwork().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
#For test, put to 2000 later
num_epochs = 2000
#Reset test_loss_plot
test_loss_plot = list()
list_test_loss_plot = list()

# Train the model
for epoch in range(num_epochs): 
  
    for i, (inputs, targets) in enumerate(train_loader):
      inputs = inputs.cuda()
      targets = targets.cuda()
      # Forward pass
      outputs = model(inputs)
      loss = criterion(outputs, targets.squeeze())
        
      # Backward and optimize
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()
                
    if (epoch+1) % 50 == 0:
      print ('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))
        
    test_loss_plot.append(loss.item())
    epochs_count = [i for i in range(num_epochs)]
list_test_loss_plot.append(test_loss_plot)


In [ ]:
# Plot the graph, find a better graph, optimally the loss aginst epoch graph, try to integrate
plt.figure(figsize=(WIDTH,HEIGHT))
plt.plot(epochs_count, list_test_loss_plot[1] ,'green',label='Mini-batch size = 8')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend(loc="best")
plt.show()

### Mini-batch size = 16

In [ ]:
 #Test for different batch size
mbatch_size = 16
train_dataset_normal = CustomTensorDataset(tensors=(X_train, y_train), transform=None)
train_loader = torch.utils.data.DataLoader(train_dataset_normal, batch_size=mbatch_size)
# Loss and optimizer
model = NeuralNetwork().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
#For test, put to 2000 later
num_epochs = 2000
#Reset test_loss_plot
test_loss_plot = list()

# Train the model
for epoch in range(num_epochs): 
  
    for i, (inputs, targets) in enumerate(train_loader):
      inputs = inputs.cuda()
      targets = targets.cuda()
      # Forward pass
      outputs = model(inputs)
      loss = criterion(outputs, targets.squeeze())
        
      # Backward and optimize
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()
                
    if (epoch+1) % 50 == 0:
      print ('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))
        
    test_loss_plot.append(loss.item())
    epochs_count = [i for i in range(num_epochs)]
list_test_loss_plot.append(test_loss_plot)


In [ ]:
# Plot the graph, find a better graph, optimally the loss aginst epoch graph, try to integrate
plt.figure(figsize=(WIDTH,HEIGHT))
plt.plot(epochs_count, list_test_loss_plot[2] ,'green',label='mbs = 16')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend(loc="best")
plt.show()

### Mini-batch size = 32

In [ ]:
 #Test for different batch size
mbatch_size = 32
train_dataset_normal = CustomTensorDataset(tensors=(X_train, y_train), transform=None)
train_loader = torch.utils.data.DataLoader(train_dataset_normal, batch_size=mbatch_size)
# Loss and optimizer
model = NeuralNetwork().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
#For test, put to 2000 later
num_epochs = 2000
#Reset test_loss_plot
test_loss_plot = list()

# Train the model
for epoch in range(num_epochs): 
  
    for i, (inputs, targets) in enumerate(train_loader):
      inputs = inputs.cuda()
      targets = targets.cuda()
      # Forward pass
      outputs = model(inputs)
      loss = criterion(outputs, targets.squeeze())
        
      # Backward and optimize
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()
                
    if (epoch+1) % 50 == 0:
      print ('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))
        
    test_loss_plot.append(loss.item())
    epochs_count = [i for i in range(num_epochs)]
list_test_loss_plot.append(test_loss_plot)

In [ ]:
# Plot the graph, find a better graph, optimally the loss aginst epoch graph, try to integrate
plt.figure(figsize=(WIDTH,HEIGHT))
plt.plot(epochs_count, list_test_loss_plot[3] ,'green',label='mbs = 32')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend(loc="best")
plt.show()

### Mini-batch size = 64

In [ ]:
 #Test for different batch size
mbatch_size = 64
train_dataset_normal = CustomTensorDataset(tensors=(X_train, y_train), transform=None)
train_loader = torch.utils.data.DataLoader(train_dataset_normal, batch_size=mbatch_size)
# Loss and optimizer
model = NeuralNetwork().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
#For test, put to 2000 later
num_epochs = 2000
#Reset test_loss_plot
test_loss_plot = list()

# Train the model
for epoch in range(num_epochs): 
  
    for i, (inputs, targets) in enumerate(train_loader):
      inputs = inputs.cuda()
      targets = targets.cuda()
      # Forward pass
      outputs = model(inputs)
      loss = criterion(outputs, targets.squeeze())
        
      # Backward and optimize
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()
                
    if (epoch+1) % 50 == 0:
      print ('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))
        
    test_loss_plot.append(loss.item())
    epochs_count = [i for i in range(num_epochs)]
list_test_loss_plot.append(test_loss_plot)


In [ ]:
# Plot the graph, find a better graph, optimally the loss aginst epoch graph, try to integrate
plt.figure(figsize=(WIDTH,HEIGHT))
plt.plot(epochs_count, list_test_loss_plot[4] ,'green',label='mbs = 64')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend(loc="best")
plt.show()

### Comparing all different mini-batch size against test loss

In [ ]:
# cross validation accuracy of different hidden neuron against epochs
plt.figure(figsize=(WIDTH,HEIGHT))
plt.title("Loss vs Epochs")
plt.plot(epochs_count,list_test_loss_plot[0],'yellow',label='mbs = 8')
plt.plot(epochs_count,list_test_loss_plot[1],'orange',label='mbs = 16')
plt.plot(epochs_count,list_test_loss_plot[2],'green',label='mbs = 32')
plt.plot(epochs_count,list_test_loss_plot[3],'blue',label='mbs = 64')
plt.plot(epochs_count,list_test_loss_plot[4],'red',label='mbs = 128')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend(loc="best")